In [1]:
import talib
import numpy as np
import pandas as pd
import random

In [2]:
file_path = '/Users/mymac/Google_Drive/Forex_Robot/'

In [3]:
df = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2020-2021.csv')
df.Date = pd.to_datetime(df.Date)

df.reset_index(drop=True, inplace=True)

In [4]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2020-07-30 07:00:00   1.29543   1.29607  1.29540    1.29572   1.29562   
1 2020-07-30 07:05:00   1.29569   1.29591  1.29505    1.29542   1.29588   
2 2020-07-30 07:10:00   1.29541   1.29590  1.29514    1.29559   1.29557   
3 2020-07-30 07:15:00   1.29561   1.29567  1.29510    1.29515   1.29577   
4 2020-07-30 07:20:00   1.29514   1.29643  1.29501    1.29642   1.29529   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  
0   1.29623  1.29560    1.29589   1.29552   1.29615  1.29550    1.29580  
1   1.29608  1.29522    1.29560   1.29578   1.29600  1.29514    1.29551  
2   1.29606  1.29535    1.29575   1.29549   1.29598  1.29524    1.29567  
3   1.29584  1.29530    1.29533   1.29569   1.29576  1.29520    1.29524  
4   1.29660  1.29518    1.29660   1.29522   1.29652  1.29510    1.29651  
---------------------------------------------------------------------------
                     Date  Bi

In [5]:
df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Mid_Close'])
df['slowk'], df['slowd'] = talib.STOCH(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['ema200'] = talib.EMA(df['Mid_Close'], timeperiod=200)
df['ema100'] = talib.EMA(df['Mid_Close'], timeperiod=100)
df['ema50'] = talib.EMA(df['Mid_Close'], timeperiod=50)
df['ema25'] = talib.EMA(df['Mid_Close'], timeperiod=25)
df['sar'] = talib.SAR(df['Mid_High'], df['Mid_Low'], acceleration=0.02, maximum=0.2)
df['atr'] = talib.ATR(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=14)
df['rsi'] = talib.RSI(df['Mid_Close'], timeperiod=14)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [6]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]

In [7]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee


def run_simulation(risk_reward_ratio, spread_cutoff, use_tl, min_pips_to_risk, max_pips_to_risk):
    max_pips_to_risk /= 10000
    min_pips_to_risk /= 10000
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    n_units = 50000
    trade = None
    lookback = 12

    i = lookback

    while i < len(df):
        curr_date = df.loc[df.index[i], 'Date']
        ema200_2, ema100_2, ema50_2, ema25_2, atr2, sar2, rsi2, mid_open2, mid_close2 = df.loc[df.index[i - 2], ['ema200', 'ema100', 'ema50', 'ema25', 'atr', 'sar', 'rsi', 'Mid_Open', 'Mid_Close']]
        ema200_1, ema100_1, ema50_1, ema25_1, atr1, sar1, rsi1, mid_open1, mid_close1 = df.loc[df.index[i - 1], ['ema200', 'ema100', 'ema50', 'ema25', 'atr', 'sar', 'rsi', 'Mid_Open', 'Mid_Close']]
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        enough_volatility = spread <= spread_cutoff
        macd2, macdsignal2 = df.loc[df.index[i - 2], ['macd', 'macdsignal']]
        macd1, macdsignal1 = df.loc[df.index[i - 1], ['macd', 'macdsignal']]
        macd_vals = [0, macd2, macdsignal2, macd1, macdsignal1]
        curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
          df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                              'Ask_Low', 'Ask_Close']]
        # emas_buy_signal = ema200_2 < ema25_2 and ema200_1 < ema25_1 and ema25_2 < ema25_1
        # emas_sell_signal = ema200_2 > ema25_2 and ema200_1 > ema25_1 and ema25_2 > ema25_1
        emas_buy_signal = ema200_2 < ema50_2 and ema200_1 < ema50_1 and ema50_2 < ema50_1 and ema25_2 < ema25_1
        emas_sell_signal = ema200_2 > ema50_2 and ema200_1 > ema50_1 and ema50_2 > ema50_1 and ema25_2 > ema25_1
        # emas_buy_signal = ema200_2 < ema50_2 and ema200_1 < ema50_1 and ema50_2 < ema25_2 and ema50_1 < ema25_1
        # emas_sell_signal = ema200_2 > ema50_2 and ema200_1 > ema50_1 and ema50_2 > ema25_2 and ema50_1 > ema25_1
        # emas_buy_signal = ema200_2 < ema25_2 and ema200_1 < ema25_1  and min_rsi <= 30
        # emas_sell_signal = ema200_2 > ema25_2 and ema200_1 > ema25_1 and max_rsi >= 70
        atr_vals = [atr2, atr1]
        abs_macd_vals = [abs(macd2), abs(macdsignal2), abs(macd1), abs(macdsignal1)]
        # atr_signal = max(abs_macd_vals) >= math.floor(min(atr_vals) * 10000) / 10000
        atr_signal = max(abs_macd_vals) >= min(atr_vals) / 2
        # atr_signal = True

        if trade is None:
          if macd2 < macdsignal2 and macd1 > macdsignal1 and max(macd_vals) == 0 and emas_buy_signal and sar2 < min([mid_open2, mid_close2]) and sar1 < min([mid_open1, mid_close1]) and enough_volatility and atr_signal:
            candle_below_ema200 = True

            # for z in range(i - 4, i):
            #   mid_high, mid_low, curr_ema200 = df.loc[df.index[z], ['Mid_High', 'Mid_Low', 'ema200']]

            #   if min([mid_high, mid_low]) < curr_ema200:
            #     candle_below_ema200 = True
            #     break

            if candle_below_ema200:
              all_buys = True

              if all_buys:
                open_price = float(curr_ask_open)
                pullback = None
                pullback = min(df.loc[df.index[i - lookback:i], 'Mid_Low'])

                if open_price - pullback < min_pips_to_risk:
                  pullback = open_price - min_pips_to_risk

                elif open_price - pullback > max_pips_to_risk:
                  pullback = open_price - max_pips_to_risk

                if pullback is not None:
                    stop_loss = round(pullback, 5)
                    # stop_loss = round(pullback - pullback_cushion, 3)

                    if stop_loss < open_price:
                        curr_pips_to_risk = open_price - stop_loss

                        if curr_pips_to_risk <= max_pips_to_risk:
                            stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                            # stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 3)

                            trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                              'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                              'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                            n_buys += 1

                            pips_risked.append(curr_pips_to_risk)

          elif macd2 > macdsignal2 and macd1 < macdsignal1 and min(macd_vals) == 0 and emas_sell_signal and sar2 > max([mid_open2, mid_close2]) and sar1 > max([mid_open1, mid_close1]) and enough_volatility and atr_signal:
            candle_above_ema200 = True
            
            # for z in range(i - 4, i):
            #   mid_high, mid_low, curr_ema200 = df.loc[df.index[z], ['Mid_High', 'Mid_Low', 'ema200']]

            #   if max([mid_high, mid_low]) > curr_ema200:
            #     candle_above_ema200 = True
            #     break

            if candle_above_ema200:
              all_sells = True

              if all_sells:
                open_price = float(curr_bid_open)
                pullback = None
                pullback = max(df.loc[df.index[i - lookback:i], 'Mid_High'])

                if pullback - open_price < min_pips_to_risk:
                  pullback = open_price + min_pips_to_risk

                elif pullback - open_price > max_pips_to_risk:
                  pullback = open_price + max_pips_to_risk

                if pullback is not None:
                    stop_loss = round(pullback, 5)
                    # stop_loss = round(pullback + pullback_cushion, 3)

                    if stop_loss > open_price:
                        curr_pips_to_risk = stop_loss - open_price

                        if curr_pips_to_risk <= max_pips_to_risk:
                            stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                            # stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 3)

                            trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                    'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                    'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                            n_sells += 1

                            pips_risked.append(curr_pips_to_risk)

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
          trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if use_tl:
          if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high - trade['pips_risked'] > trade['stop_loss']:
              trade['stop_loss'] = curr_bid_high - trade['pips_risked']
          # if trade is not None and trade['original_units'] == trade['n_units'] and trade['trade_type'] == 'buy' and curr_bid_high - trade['open_price'] >= trade['pips_risked']:
          #     trade['stop_loss'] = trade['open_price']
          #     reward += trade['pips_risked'] * (trade['n_units'] / 2) * value_per_pip
          #     day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'] / 2)
          #     trade['n_units'] = trade['n_units'] / 2
              

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
          trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
          trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if use_tl:
          if trade is not None and trade['trade_type'] == 'sell' and trade['pips_risked'] + curr_ask_low < trade['stop_loss']:
            trade['stop_loss'] = trade['pips_risked'] + curr_ask_low
          # if trade is not None and trade['original_units'] == trade['n_units'] and trade['trade_type'] == 'sell' and trade['open_price'] - curr_ask_low >= trade['pips_risked']:
          #     trade['stop_loss'] = trade['open_price']
          #     reward += trade['pips_risked'] * (trade['n_units'] / 2) * value_per_pip
          #     day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'] / 2)
          #     trade['n_units'] = trade['n_units'] / 2

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
          trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        i += 1

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [8]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.5, 2]
# risk_reward_ratio_vals = [2]
spread_cutoffs = [2, 2.5]
# spread_cutoffs = [2.5]
use_tl_vals = [True, False]
# use_tl_vals = [False]
min_pips_to_risk_vals = [5, 10]
# min_pips_to_risk_vals = [10]
max_pips_to_risk_vals = [15, 20, 25, 30]
# max_pips_to_risk_vals = [30]
n_possibilities = len(risk_reward_ratio_vals) * len(spread_cutoffs) * len(use_tl_vals) * len(min_pips_to_risk_vals) * len(max_pips_to_risk_vals)
all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
  for spread_val in spread_cutoffs:
    for use_tl in use_tl_vals:
      for min_pips in min_pips_to_risk_vals:
        for max_pips in max_pips_to_risk_vals:
          all_combos.append((risk_reward_ratio, spread_val, use_tl, min_pips, max_pips))

# percentage_to_try = 0.5
percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_use_tl_val = None
best_min_pips_val = None
best_max_pips_val = None
top_n_results = 20
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, use_tl, min_pips, max_pips in combos_to_try:
  reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_simulation(risk_reward_ratio, spread_val, use_tl, min_pips, max_pips)
  runs_finished += 1

  print(reward)
  print('Num buys: ' + str(n_sells))
  print('Num sells: ' + str(n_buys))
  print('Num trades: ' + str(n_buys + n_sells))
  print('Num wins: ' + str(n_wins))
  print('Num losses: ' + str(n_losses))
  print('Win streak: ' + str(win_streak))
  print('Loss streak: ' + str(loss_streak))
  # print('Avg pips risked: ' + str(sum(pips_risked) / len(pips_risked)))
  print('Remaining runs: ' + str(n_runs - runs_finished))

  min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None
  if min_item is None or reward > min_item['reward']:
    if min_item is not None:
      best_rewards.remove(min_item)
    best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread': spread_val, 'use_tl': use_tl, 'min_pips': min_pips, 'max_pips': max_pips})


  if reward > best_reward:
    best_reward = reward
    best_risk_reward = risk_reward_ratio
    best_spread_cutoff = spread_val
    best_use_tl_val = use_tl
    best_min_pips_val = min_pips
    best_max_pips_val = max_pips

  print('Best reward so far: ' + str(best_reward))
  print()

Num runs: 64

-95.99999999991351
Num buys: 19
Num sells: 16
Num trades: 35
Num wins: 13
Num losses: 22
Win streak: 4
Loss streak: 9
Remaining runs: 63
Best reward so far: -95.99999999991351

-268.99999999988506
Num buys: 27
Num sells: 25
Num trades: 52
Num wins: 15
Num losses: 37
Win streak: 3
Loss streak: 15
Remaining runs: 62
Best reward so far: -95.99999999991351

-447.4999999999392
Num buys: 20
Num sells: 16
Num trades: 36
Num wins: 11
Num losses: 25
Win streak: 3
Loss streak: 13
Remaining runs: 61
Best reward so far: -95.99999999991351

-139.49999999991184
Num buys: 19
Num sells: 16
Num trades: 35
Num wins: 13
Num losses: 22
Win streak: 4
Loss streak: 9
Remaining runs: 60
Best reward so far: -95.99999999991351

-319.4999999998937
Num buys: 27
Num sells: 26
Num trades: 53
Num wins: 16
Num losses: 37
Win streak: 3
Loss streak: 15
Remaining runs: 59
Best reward so far: -95.99999999991351

1395.000000000067
Num buys: 27
Num sells: 26
Num trades: 53
Num wins: 22
Num losses: 31
Win stre

In [ ]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best TL val: ' + str(best_use_tl_val))
print('Best min pips to risk: ' + str(best_min_pips_val))
print('Best max pips to risk: ' + str(best_max_pips_val))
print('Best atr multiplier: ' + str(best_macd_cutoff))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 210.99999999999852
Best risk/reward ratio: 2
Best spread: 2
Best TL val: False
Best min pips to risk: 10
Best max pips to risk: 25
Best atr multiplier: 0.0005
-----------------------
Top results:
{'reward': 209, 'ratio': 2, 'spread': 2, 'use_tl': False, 'min_pips': 5, 'max_pips': 25, 'macd_cutoff': 0.0004}
{'reward': 209, 'ratio': 2, 'spread': 2.5, 'use_tl': False, 'min_pips': 10, 'max_pips': 25, 'macd_cutoff': 0.0004}
{'reward': 209, 'ratio': 2, 'spread': 2.5, 'use_tl': False, 'min_pips': 5, 'max_pips': 25, 'macd_cutoff': 0.0004}
{'reward': 208, 'ratio': 2, 'spread': 2, 'use_tl': False, 'min_pips': 10, 'max_pips': 40, 'macd_cutoff': 0.0005}
{'reward': 210, 'ratio': 2, 'spread': 2, 'use_tl': False, 'min_pips': 10, 'max_pips': 25, 'macd_cutoff': 0.0005}
{'reward': 208, 'ratio': 2, 'spread': 2, 'use_tl': False, 'min_pips': 5, 'max_pips': 25, 'macd_cutoff': 0.0005}
{'reward': 210, 'ratio': 2, 'spread': 2, 'use_tl': False, 'min_pips': 10

In [ ]:
# {'reward': 409, 'ratio': 2, 'spread': 2, 'use_tl': False, 'min_pips': 10, 'max_pips': 20, 'use_atr_multiplier': 2}
# {'reward': 185, 'ratio': 2, 'spread': 2, 'use_tl': False, 'min_pips': 5, 'max_pips': 20, 'use_atr_multiplier': 2}
# {'reward': 149, 'ratio': 2, 'spread': 2, 'use_tl': True, 'min_pips': 10, 'max_pips': 20, 'use_atr_multiplier': 2}
# {'reward': 406, 'ratio': 2, 'spread': 2.5, 'use_tl': False, 'min_pips': 10, 'max_pips': 20, 'use_atr_multiplier': 2}
# {'reward': 11, 'ratio': 2, 'spread': 2, 'use_tl': False, 'min_pips': 5, 'max_pips': 20, 'use_atr_multiplier': 1.5}
# {'reward': 22, 'ratio': 2, 'spread': 2.5, 'use_tl': True, 'min_pips': 5, 'max_pips': 20, 'use_atr_multiplier': 1.5}
# {'reward': -88, 'ratio': 1.5, 'spread': 2.5, 'use_tl': True, 'min_pips': 10, 'max_pips': 20, 'use_atr_multiplier': 1.5}
# {'reward': -61, 'ratio': 1.5, 'spread': 2.5, 'use_tl': False, 'min_pips': 10, 'max_pips': 20, 'use_atr_multiplier': 2}
# {'reward': 147, 'ratio': 2, 'spread': 2.5, 'use_tl': False, 'min_pips': 10, 'max_pips': 20, 'use_atr_multiplier': 1.5}
# {'reward': 7, 'ratio': 2, 'spread': 2.5, 'use_tl': False, 'min_pips': 5, 'max_pips': 20, 'use_atr_multiplier': 1.5}
# {'reward': 23, 'ratio': 2, 'spread': 2, 'use_tl': True, 'min_pips': 5, 'max_pips': 20, 'use_atr_multiplier': 1.5}
# {'reward': 150, 'ratio': 2, 'spread': 2, 'use_tl': False, 'min_pips': 10, 'max_pips': 20, 'use_atr_multiplier': 1.5}
# {'reward': 57, 'ratio': 2, 'spread': 2.5, 'use_tl': True, 'min_pips': 5, 'max_pips': 20, 'use_atr_multiplier': 2}
# {'reward': -90, 'ratio': 1.5, 'spread': 2, 'use_tl': True, 'min_pips': 10, 'max_pips': 20, 'use_atr_multiplier': 1.5}
# {'reward': 150, 'ratio': 2, 'spread': 2.5, 'use_tl': True, 'min_pips': 10, 'max_pips': 20, 'use_atr_multiplier': 2}
# {'reward': 161, 'ratio': 2, 'spread': 2.5, 'use_tl': True, 'min_pips': 10, 'max_pips': 20, 'use_atr_multiplier': 1.5}
# {'reward': 161, 'ratio': 2, 'spread': 2, 'use_tl': True, 'min_pips': 10, 'max_pips': 20, 'use_atr_multiplier': 1.5}
# {'reward': 57, 'ratio': 2, 'spread': 2, 'use_tl': True, 'min_pips': 5, 'max_pips': 20, 'use_atr_multiplier': 2}
# {'reward': -65, 'ratio': 1.5, 'spread': 2, 'use_tl': False, 'min_pips': 10, 'max_pips': 20, 'use_atr_multiplier': 2}
# {'reward': 184, 'ratio': 2, 'spread': 2.5, 'use_tl': False, 'min_pips': 5, 'max_pips': 20, 'use_atr_multiplier': 2}


# {'reward': -51, 'ratio': 2, 'spread': 2, 'use_tl': False, 'min_pips': 10, 'max_pips': 15, 'use_atr_multiplier': 2}
# {'reward': -34, 'ratio': 2, 'spread': 2, 'use_tl': True, 'min_pips': 10, 'max_pips': 20, 'use_atr_multiplier': 2}
# {'reward': 355, 'ratio': 2, 'spread': 2, 'use_tl': False, 'min_pips': 10, 'max_pips': 20, 'use_atr_multiplier': 1.5}
# {'reward': 241, 'ratio': 2, 'spread': 2.5, 'use_tl': False, 'min_pips': 10, 'max_pips': 15, 'use_atr_multiplier': 1.5}
# {'reward': -53, 'ratio': 2, 'spread': 2.5, 'use_tl': False, 'min_pips': 10, 'max_pips': 15, 'use_atr_multiplier': 2}
# {'reward': -70, 'ratio': 2, 'spread': 2.5, 'use_tl': True, 'min_pips': 5, 'max_pips': 20, 'use_atr_multiplier': 2}
# {'reward': 127, 'ratio': 2, 'spread': 2.5, 'use_tl': False, 'min_pips': 5, 'max_pips': 15, 'use_atr_multiplier': 1.5}
# {'reward': 351, 'ratio': 2, 'spread': 2.5, 'use_tl': False, 'min_pips': 10, 'max_pips': 20, 'use_atr_multiplier': 1.5}
# {'reward': 128, 'ratio': 2, 'spread': 2, 'use_tl': False, 'min_pips': 5, 'max_pips': 15, 'use_atr_multiplier': 1.5}
# {'reward': -37, 'ratio': 2, 'spread': 2.5, 'use_tl': True, 'min_pips': 10, 'max_pips': 20, 'use_atr_multiplier': 2}
# {'reward': 380, 'ratio': 2, 'spread': 2.5, 'use_tl': False, 'min_pips': 5, 'max_pips': 20, 'use_atr_multiplier': 2}
# {'reward': 595, 'ratio': 2, 'spread': 2.5, 'use_tl': False, 'min_pips': 10, 'max_pips': 20, 'use_atr_multiplier': 2}
# {'reward': 375, 'ratio': 2, 'spread': 2, 'use_tl': False, 'min_pips': 5, 'max_pips': 20, 'use_atr_multiplier': 2}
# {'reward': 591, 'ratio': 2, 'spread': 2, 'use_tl': False, 'min_pips': 10, 'max_pips': 20, 'use_atr_multiplier': 2}
# {'reward': 236, 'ratio': 2, 'spread': 2, 'use_tl': False, 'min_pips': 5, 'max_pips': 20, 'use_atr_multiplier': 1.5}
# {'reward': -22, 'ratio': 2, 'spread': 2.5, 'use_tl': True, 'min_pips': 10, 'max_pips': 20, 'use_atr_multiplier': 1.5}
# {'reward': 243, 'ratio': 2, 'spread': 2, 'use_tl': False, 'min_pips': 10, 'max_pips': 15, 'use_atr_multiplier': 1.5}
# {'reward': 235, 'ratio': 2, 'spread': 2.5, 'use_tl': False, 'min_pips': 5, 'max_pips': 20, 'use_atr_multiplier': 1.5}
# {'reward': -27, 'ratio': 2, 'spread': 2, 'use_tl': True, 'min_pips': 10, 'max_pips': 20, 'use_atr_multiplier': 1.5}
# {'reward': -69, 'ratio': 2, 'spread': 2, 'use_tl': True, 'min_pips': 5, 'max_pips': 20, 'use_atr_multiplier': 2}

In [89]:
import math

In [94]:
foo = 0.00040

math.floor(foo * 10000) / 10000

0.0004